<a href="https://colab.research.google.com/github/peregilk/VACMA/blob/master/EPFL_Vaccine_Tweet_Remove_Duplicates.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Vaccine Tweet Remove Duplicates


In [0]:
#@title Initiate and Load the Data from Google Sheet
#@markdown To run this script you will need a Google sheet with all the tweets that needs to be preprocessed. 

!pip install --upgrade --quiet gspread
import os, sys, glob
import numpy as np
import gspread
import pandas as pd

from google.colab import drive
from google.colab import auth
auth.authenticate_user()

from oauth2client.client import GoogleCredentials
gc = gspread.authorize(GoogleCredentials.get_application_default())

spreadsheet_name = "EPFL unanotated tweets"#@param {type:"string"}

#@markdown <br /><br /> Leave the sheet name open if you want to analyse the first sheet in the spreadsheet.
sheet_name="raw"#@param {type:"string"}

#@markdown <br /><br /> The target document needs a column with an ID for every tweet. This ID will be used to link back when creating the Unique tweets
ID_column_number=1  #@param {type:"integer"}

#@markdown <br /><br /> Please insert the column number that contains the data that should be checked for duplicates.  The script assumes that the first row is a header, and will not touch this.

source_column_number=2  #@param {type:"integer"}
first_row_header = True
##@param {type:"boolean"}
#@markdown <br /><br /> Other columns that should be carried over into the unique set. This is typically columns with date information, previous completed codings etc. The original value will be passed along. Please note that if you would like to preserve the date from the first post, you should sort the original spreadsheet by date before running this script. The expected input is a comma-separated list of numbers.

duplicate_column_numbers=""  #@param {type:"string"}

if duplicate_column_numbers:
  include_columns = duplicate_column_numbers.split(",")
  include_columns = list(map(int, include_columns)) 
else:
  include_columns = []

#Open the spreadsheet
try:
  spreadsheet = gc.open(spreadsheet_name)
  #print("Spreadsheet is opened")
except:
  print ("An error occured while trying to open the spreadsheet. Does not seem to be a spreadwheet with this name in your home folder.")
  sys.exit()

#Open the worksheet
try:
  if sheet_name:
    worksheet = spreadsheet.worksheet(sheet_name)
  else:
    worksheet = spreadsheet.get_worksheet(0)
  #print("Worksheet successfully selected")

except:
  print("Unable to select worksheet")
  sys.exit()

#Get all values
values = worksheet.get_all_values()

if(len(values) > 0):
  print("Successfully read "+str(len(values)-1)+" items")
  if(first_row_header):
    print("ID Column Header: \""+values[0][ID_column_number-1]+"\"")
    print("Target Column Header: \""+values[0][source_column_number-1]+"\"")
    print("Target First data row: \""+values[1][source_column_number-1]+"\"")
    print("Also include the following "+str(len(include_columns))+" columns: ")
    for c in include_columns:
      print("     "+values[0][c-1])

else:
  print("No valid posts. Please check the input variables.")    
#@markdown <br /><br /> Do a test run where you just run this cell. Make sure that the input is OK. Running this cell will not alter any data.

In [0]:
#@title Create Sheet with Unique Tweets
!pip install --upgrade --quiet gspread
!pip install python-Levenshtein
import Levenshtein
import pandas as pd
from datetime import datetime
from datetime import timedelta


def reauthenticate():
  gc = gspread.authorize(GoogleCredentials.get_application_default())
  spreadsheet = gc.open(spreadsheet_name)
  newworksheet = spreadsheet.worksheet(unique_sheet_name)
  return newworksheet

program_start_time=datetime.now()


#Loop through the dataset and find duplicates
maxDistance = 10 #@param {type:"integer"}
#@markdown <br /><br />Adjustment for short texts. Protects short tweets so that Levenshtein-distance is relatively shorter for short tweets. The default setting is "10"

shortAdjust = 10 #@param {type:"integer"}

#Set this to 1 to drop headers - default
# #@markdown <br /><br />The default value here is "1". If the dataset is huge, it might however be necessary to restart the script. Then enter the value where the script should start.

start = 1 
# #@param {type:"integer"}



#@markdown <br /><br />Set the name of the worksheet where the results will be created
unique_sheet_name = "deduplicated" #@param {type:"string"}


data = pd.DataFrame.from_records(values)
data.columns = data.iloc[0]
data = data.reindex(data.index.drop(0))

try:
  newworksheet = spreadsheet.add_worksheet(title=unique_sheet_name, rows=data.shape[0], cols=data.shape[1]+len(include_columns)-1)
  print("Created new worksheet")
except:
  newworksheet = spreadsheet.worksheet(unique_sheet_name)
  print("Worksheet already created, opening existing worksheet. Starting at the end")
  start = len(newworksheet.get_all_values())-1
  if start <= 1:
    start = 1


end = data.shape[0]

#Print headers
newworksheet.update_cell(1,1,"Original ID")
newworksheet.update_cell(1,2,"DeDupe ID")
newworksheet.update_cell(1,3,"Duplicate List")
newworksheet.update_cell(1,4,"Number of Duplicates")
newworksheet.update_cell(1,5,"Is a duplicate")
y = 6
for c in include_columns:
  newworksheet.update_cell(1,y,values[0][c-1])
  y += 1
  
newworksheet.update_cell(1,y,values[0][source_column_number-1])

for i in range(start,end):
  x = data.iloc[i-1,source_column_number-1]
  id = data.iloc[i-1,ID_column_number-1]
  
  print("Analyzing #"+str(i))
  isADuplicate = 1
  firstpost = 9999999
  duplist = []
  
  for j in range(1,end):
    y = data.iloc[j-1,source_column_number-1]
    minxy = min(len(x), len(y))
    distance = Levenshtein.distance(x, y)
    if (x == y) or (distance <= maxDistance and (minxy - (minxy+ shortAdjust)/2)  > distance):
      duplist.append(data.iloc[j-1, ID_column_number-1])
      if(j-1 < firstpost):
        firstpost = j-1



  if duplist[0] == id:
    isADuplicate = 0
    
  #Reauthenticate every ten minutes 
  if datetime.now() > program_start_time + timedelta(minutes=5):
    newworksheet = reauthenticate()
    program_start_time = datetime.now()   #Resets the Program Start Time
    print("Reauthenticating at:" + str(datetime.now()))

  cell_values = []
  cell_values.append(id)
  cell_values.append(firstpost+1)
  cell_values.append(str(duplist))
  cell_values.append(int(len(duplist)))
  cell_values.append(isADuplicate)
  
  y = 6
  for c in include_columns:
    print(data.loc[i][c-1])
    cell_values.append(data.loc[i][c-1])
    y += 1

  cell_values.append(x)

  cell_list = newworksheet.range(i+1,1,i+1,y)
  for p, val in enumerate(cell_values):  #gives us a tuple of an index and value
    cell_list[p].value = val    #use the index on cell_list and the val from cell_values

  newworksheet.update_cells(cell_list)
  